In [2]:
from melusine.nlp_tools.embedding import Embedding
import pandas as pd
from melusine.nlp_tools.phraser import Phraser, phraser_on_body
from melusine.nlp_tools.tokenizer import Tokenizer
from sklearn.pipeline import Pipeline
from melusine.utils.transformer_scheduler import TransformerScheduler
from melusine.models.neural_architectures import cnn_model, rnn_model, transformers_model, bert_model
from melusine.models.train import NeuralModel
from sklearn import metrics
from sklearn.model_selection import train_test_split, ShuffleSplit
import numpy as np
import tqdm
tqdm.pandas()

In [4]:
data = pd.read_pickle('data/mails_lemmatized.pkl')
df_email = pd.DataFrame()
df_email['text'] = [' '.join(words) for words in data]
df_email['label'] = pd.read_pickle('data/mail_labels.pkl')
df_email = df_email[df_email['label']!=2]
df_email.head()

,text,label
0,1ere relecture gt consommation yohan temps c c...,0
1,accepter accepter demande cliquez simplement b...,0
2,acce decibel bonjour pouvoir donner acces deci...,1
3,acces rec4 decibel sdw rec4 hm dm ad restituti...,1
4,acte indemnite hospitalier savoir c acte n fra...,1


In [41]:
shuffler = ShuffleSplit(n_splits=1,test_size=0.15)
train_indexes, test_indexes = list(shuffler.split(df_email.index))[0]

In [ ]:
def tokenize(row):
    tokens = row.split(' ')
    return tokens 

tknz = Tokenizer('text')
df_email2 = tknz.fit_transform(df_email)

In [51]:
df_train = df_email.iloc[train_indexes]
df_test = df_email.iloc[test_indexes]

In [52]:
embedding = Embedding(input_column='text', min_count=5)
embedding.train(df_train)

# CNN

In [64]:
cnn_nn = NeuralModel(architecture_function=cnn_model,
                       pretrained_embedding=embedding,
                       text_input_column='text')

In [55]:
cnn_nn.fit(df_test.drop('label', axis = 1), df_train.label)

C:\Users\le_paumier-m\Anaconda3\lib\site-packages\melusine\nlp_tools\tokenizer.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["tokens"] = apply_func(X, self.tokenize)
C:\Users\le_paumier-m\Anaconda3\lib\site-packages\melusine\nlp_tools\tokenizer.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["tokens"] = apply_func(X, lambda x: x["tokens"][0])


Epoch 1/15
1/1 [==============================] - 5s 5s/step - loss: 0.9173 - accuracy: 0.3930
Epoch 2/15
1/1 [==============================] - 4s 4s/step - loss: 0.4576 - accuracy: 0.8570
Epoch 3/15
1/1 [==============================] - 4s 4s/step - loss: 0.4950 - accuracy: 0.8581
Epoch 4/15
1/1 [==============================] - 4s 4s/step - loss: 0.4709 - accuracy: 0.8493
Epoch 5/15
1/1 [==============================] - 4s 4s/step - loss: 0.4574 - accuracy: 0.8328
Epoch 6/15
1/1 [==============================] - 4s 4s/step - loss: 0.4516 - accuracy: 0.8292
Epoch 7/15
1/1 [==============================] - 4s 4s/step - loss: 0.4220 - accuracy: 0.8526
Epoch 8/15
1/1 [==============================] - 4s 4s/step - loss: 0.4167 - accuracy: 0.8589
Epoch 9/15
1/1 [==============================] - 4s 4s/step - loss: 0.4124 - accuracy: 0.8607
Epoch 10/15
1/1 [==============================] - 4s 4s/step - loss: 0.3952 - accuracy: 0.8600
Epoch 11/15
1/1 [==============================] 

In [60]:
y_pred = cnn_nn.predict(df_test.drop('label', axis = 1))
CM = metrics.confusion_matrix(df_test.label, y_pred)
print(CM)
FN = CM[1,0]
print(FN/np.sum(CM))

[[409   0]
 [ 73   0]]
0.15145228215767634


# RNN

In [68]:
rnn_nn = NeuralModel(architecture_function=rnn_model,
                       pretrained_embedding=embedding,
                       text_input_column='text')

In [70]:
rnn_nn.fit(df_train.drop('label', axis = 1), df_train.label)

Epoch 1/15
1/1 [==============================] - 12s 12s/step - loss: 0.5341 - accuracy: 0.8405
Epoch 2/15
1/1 [==============================] - 7s 7s/step - loss: 0.4068 - accuracy: 0.8600
Epoch 3/15
1/1 [==============================] - 7s 7s/step - loss: 0.4394 - accuracy: 0.8600
Epoch 4/15
1/1 [==============================] - 7s 7s/step - loss: 0.4325 - accuracy: 0.8600
Epoch 5/15
1/1 [==============================] - 7s 7s/step - loss: 0.4097 - accuracy: 0.8600
Epoch 6/15
1/1 [==============================] - 7s 7s/step - loss: 0.3960 - accuracy: 0.8600
Epoch 7/15
1/1 [==============================] - 8s 8s/step - loss: 0.3923 - accuracy: 0.8600
Epoch 8/15
1/1 [==============================] - 7s 7s/step - loss: 0.4020 - accuracy: 0.8600
Epoch 9/15
1/1 [==============================] - 7s 7s/step - loss: 0.3928 - accuracy: 0.8600
Epoch 10/15
1/1 [==============================] - 7s 7s/step - loss: 0.3829 - accuracy: 0.8600
Epoch 11/15
1/1 [==============================

In [71]:
y_pred = rnn_nn.predict(df_test.drop('label', axis = 1))
CM = metrics.confusion_matrix(df_test.label, y_pred)
print(CM)
FN = CM[1,0]
print(FN/np.sum(CM))

[[409   0]
 [ 73   0]]
0.15145228215767634


# Transformer

In [72]:
transf_nn = NeuralModel(architecture_function=transformers_model,
                       pretrained_embedding=embedding,
                       text_input_column='text')

In [73]:
transf_nn.fit(df_train.drop('label', axis = 1), df_train.label)

Epoch 1/15
1/1 [==============================] - 52s 52s/step - loss: 2.9108 - accuracy: 0.1653
Epoch 2/15
1/1 [==============================] - 40s 40s/step - loss: 0.8087 - accuracy: 0.8589
Epoch 3/15
1/1 [==============================] - 32s 32s/step - loss: 0.6094 - accuracy: 0.8581
Epoch 4/15
1/1 [==============================] - 30s 30s/step - loss: 0.4710 - accuracy: 0.8402
Epoch 5/15
1/1 [==============================] - 29s 29s/step - loss: 0.4773 - accuracy: 0.8167
Epoch 6/15
1/1 [==============================] - 32s 32s/step - loss: 0.4395 - accuracy: 0.8501
Epoch 7/15
1/1 [==============================] - 34s 34s/step - loss: 0.4464 - accuracy: 0.8570
Epoch 8/15
1/1 [==============================] - 30s 30s/step - loss: 0.4468 - accuracy: 0.8600
Epoch 9/15
1/1 [==============================] - 29s 29s/step - loss: 0.4368 - accuracy: 0.8592
Epoch 10/15
1/1 [==============================] - 34s 34s/step - loss: 0.4344 - accuracy: 0.8596
Epoch 11/15
1/1 [============

In [74]:
y_pred = transf_nn.predict(df_test.drop('label', axis = 1))
CM = metrics.confusion_matrix(df_test.label, y_pred)
print(CM)
FN = CM[1,0]
print(FN/np.sum(CM))

[[409   0]
 [ 73   0]]
0.15145228215767634


# Bert

In [ ]:
bert_nn = NeuralModel(architecture_function=bert_model,
                       pretrained_embedding=embedding,
                       text_input_column='text')

In [ ]:
bert_nn.fit(df_train.drop('label', axis = 1), df_train.label)

In [ ]:
y_pred = bert_nn.predict(df_test.drop('label', axis = 1))
CM = metrics.confusion_matrix(df_test.label, y_pred)
print(CM)
FN = CM[1,0]
print(FN/np.sum(CM))